## Unpack Kaldi

In [1]:
%cd /opt

/opt


In [2]:
%%capture
!tar xvf /kaggle/input/extract-prebuilt-kaldi-from-docker/kaldi.tar

In [3]:
import os
os.environ['LD_LIBRARY_PATH'] = '/opt/conda/lib:/opt/kaldi/tools/openfst-1.6.7/lib:/opt/kaldi/src/lib'
EXISTING_PATH = os.environ['PATH']

In [4]:
%cd /


/


In [5]:
!tar xvf /kaggle/input/extract-cuda-from-kaldi-docker/cuda.tar

./usr/local/cuda-10.0/
./usr/local/cuda-10.0/bin/
./usr/local/cuda-10.0/bin/nvlink
./usr/local/cuda-10.0/bin/cuobjdump
./usr/local/cuda-10.0/bin/ptxas
./usr/local/cuda-10.0/bin/gpu-library-advisor
./usr/local/cuda-10.0/bin/bin2c
./usr/local/cuda-10.0/bin/fatbinary
./usr/local/cuda-10.0/bin/cuda-gdb
./usr/local/cuda-10.0/bin/nvprof
./usr/local/cuda-10.0/bin/nvprune
./usr/local/cuda-10.0/bin/crt/
./usr/local/cuda-10.0/bin/crt/prelink.stub
./usr/local/cuda-10.0/bin/crt/link.stub
./usr/local/cuda-10.0/bin/cudafe++
./usr/local/cuda-10.0/bin/nvcc.profile
./usr/local/cuda-10.0/bin/nvcc
./usr/local/cuda-10.0/bin/nvdisasm
./usr/local/cuda-10.0/bin/cuda-memcheck
./usr/local/cuda-10.0/bin/cuda-gdbserver
./usr/local/cuda-10.0/src/
./usr/local/cuda-10.0/src/fortran_thunking.c
./usr/local/cuda-10.0/src/fortran.c
./usr/local/cuda-10.0/src/cusparse_fortran.c
./usr/local/cuda-10.0/src/cusparse_fortran.h
./usr/local/cuda-10.0/src/fortran.h
./usr/local/cuda-10.0/src/fortran_common.h
./usr/local/cuda-10.0

In [6]:
%cd kaldi/egs

[Errno 2] No such file or directory: 'kaldi/egs'
/


## Install flac

In [7]:
%%capture
!apt install -y flac

## Create a work directory

In [8]:
!mkdir -p usels/s5
%cd usels/s5

/usels/s5


In [9]:
!mkdir /kaggle/working/data
!mkdir /kaggle/working/exp
!ln -s /kaggle/working/data
!ln -s /kaggle/working/exp

In [10]:
!ln -s ../../wsj/s5/steps
!ln -s ../../wsj/s5/utils
!ln -s ../../librispeech/s5/local

In [11]:
!mkdir conf

In [12]:
%%writefile conf/mfcc_hires.conf
# config for high-resolution MFCC features, intended for neural network training
# Note: we keep all cepstra, so it has the same info as filterbank features,
# but MFCC is more easily compressible (because less correlated) which is why 
# we prefer this method.
--use-energy=false   # use average of log energy, not energy.
--num-mel-bins=40     # similar to Google's setup.
--num-ceps=40     # there is no dimensionality reduction.
--low-freq=20     # low cutoff frequency for mel bins... this is high-bandwidth data, so
                  # there might be some information at the low end.
--high-freq=-400 # high cutoff frequently, relative to Nyquist of 8000 (=7600)

Writing conf/mfcc_hires.conf


## Setting up paths

(In the scripts, you just source `path.sh`)

In [13]:
%env KALDI_ROOT=/opt/kaldi

env: KALDI_ROOT=/opt/kaldi


In [14]:
!cat ../../wsj/s5/path.sh

cat: ../../wsj/s5/path.sh: No such file or directory


In [15]:
%env LC_ALL=C
#PWD = !pwd
PWD = '/opt/kaldi/egs/usels/s5'
KALDI_ROOT = '/opt/kaldi'
WSJ_PATH = f'{PWD}/utils/:{KALDI_ROOT}/tools/openfst/bin:{PWD}:{EXISTING_PATH}'

env: LC_ALL=C


In [16]:
!cat $KALDI_ROOT/tools/config/common_path.sh

# we assume KALDI_ROOT is already defined
[ -z "$KALDI_ROOT" ] && echo >&2 "The variable KALDI_ROOT must be already defined" && exit 1
# The formatting of the path export command is intentionally weird, because
# this allows for easy diff'ing
export PATH=\
${KALDI_ROOT}/src/bin:\
${KALDI_ROOT}/src/chainbin:\
${KALDI_ROOT}/src/featbin:\
${KALDI_ROOT}/src/fgmmbin:\
${KALDI_ROOT}/src/fstbin:\
${KALDI_ROOT}/src/gmmbin:\
${KALDI_ROOT}/src/ivectorbin:\
${KALDI_ROOT}/src/kwsbin:\
${KALDI_ROOT}/src/latbin:\
${KALDI_ROOT}/src/lmbin:\
${KALDI_ROOT}/src/nnet2bin:\
${KALDI_ROOT}/src/nnet3bin:\
${KALDI_ROOT}/src/nnetbin:\
${KALDI_ROOT}/src/online2bin:\
${KALDI_ROOT}/src/onlinebin:\
${KALDI_ROOT}/src/rnnlmbin:\
${KALDI_ROOT}/src/sgmm2bin:\
${KALDI_ROOT}/src/sgmmbin:\
${KALDI_ROOT}/src/tfrnnlmbin:\
${KALDI_ROOT}/src/cudadecoderbin:\
$PATH


In [17]:
#kaldi_paths=!cat $KALDI_ROOT/tools/config/common_path.sh|grep '/src/'|awk -F':' '{print $1}'|awk -F'/' '{print $NF}'|tr '\n' ':'
raw_kaldi_paths=!cat $KALDI_ROOT/tools/config/common_path.sh|grep '/src/'|awk -F':' '{print $1}'|awk -F'/' '{print "/opt/kaldi/src/"$NF}'

In [18]:
KALDI_PATHS=raw_kaldi_paths.nlstr.replace('\n',':')

In [19]:
!cat $KALDI_ROOT/tools/env.sh

export PATH=/opt/kaldi/tools/python:${PATH}
export PHONETISAURUS="/tmp/output/opt/kaldi/tools/phonetisaurus-g2p"
export PATH="$PATH:${PHONETISAURUS}:${PHONETISAURUS}/src/scripts"


In [20]:
PHONETISAURUS = "/tmp/output/opt/kaldi/tools/phonetisaurus-g2p"
TOOLS_PATH = f'/opt/kaldi/tools/python:{PHONETISAURUS}:{PHONETISAURUS}/src/scripts'

In [21]:
%env PATH = f"{WSJ_PATH}:{KALDI_PATHS}:{TOOLS_PATH}"

env: PATH=f"/opt/kaldi/egs/usels/s5/utils/:/opt/kaldi/tools/openfst/bin:/opt/kaldi/egs/usels/s5:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/kaldi/src/bin:/opt/kaldi/src/chainbin:/opt/kaldi/src/featbin:/opt/kaldi/src/fgmmbin:/opt/kaldi/src/fstbin:/opt/kaldi/src/gmmbin:/opt/kaldi/src/ivectorbin:/opt/kaldi/src/kwsbin:/opt/kaldi/src/latbin:/opt/kaldi/src/lmbin:/opt/kaldi/src/nnet2bin:/opt/kaldi/src/nnet3bin:/opt/kaldi/src/nnetbin:/opt/kaldi/src/online2bin:/opt/kaldi/src/onlinebin:/opt/kaldi/src/rnnlmbin:/opt/kaldi/src/sgmm2bin:/opt/kaldi/src/sgmmbin:/opt/kaldi/src/tfrnnlmbin:/opt/kaldi/src/cudadecoderbin:/opt/kaldi/tools/python:/tmp/output/opt/kaldi/tools/phonetisaurus-g2p:/tmp/output/opt/kaldi/tools/phonetisaurus-g2p/src/scripts"


In [22]:
!cat ../../wsj/s5/cmd.sh

cat: ../../wsj/s5/cmd.sh: No such file or directory


In [23]:
%env train_cmd=run.pl
%env decode_cmd="run.pl --mem 2G"

env: train_cmd=run.pl
env: decode_cmd="run.pl --mem 2G"


In [24]:
!ln -s ../../wsj/s5/cmd.sh
!ln -s ../../wsj/s5/path.sh
!ln -s utils/queue.pl
!ln -s utils/run.pl

In [25]:
!rm *.pl

## Data prep

In [26]:
!local/data_prep.sh /kaggle/input/librispeech-test-clean-and-other/LibriSpeech/test-other data/test-other
!local/data_prep.sh /kaggle/input/librispeech-test-clean-and-other/LibriSpeech/test-clean data/test-clean

/bin/bash: local/data_prep.sh: No such file or directory
/bin/bash: local/data_prep.sh: No such file or directory


In [27]:
!utils/copy_data_dir.sh data/test-clean data/test-clean_hires
!utils/copy_data_dir.sh data/test-other data/test-other_hires

/bin/bash: utils/copy_data_dir.sh: No such file or directory
/bin/bash: utils/copy_data_dir.sh: No such file or directory


In [28]:
!ln -s utils/parse_options.sh

In [29]:
!steps/make_mfcc.sh --nj 20 --mfcc-config conf/mfcc_hires.conf --cmd "$train_cmd" data/test-cleantest-clean_hires
!steps/compute_cmvn_stats.sh data/test-clean_hires
!utils/fix_data_dir.sh data/test-clean_hires
!steps/make_mfcc.sh --nj 20 --mfcc-config conf/mfcc_hires.conf --cmd "$train_cmd" data/test-other_hires
!steps/compute_cmvn_stats.sh data/test-other_hires
!utils/fix_data_dir.sh data/test-other_hires


/bin/bash: steps/make_mfcc.sh: No such file or directory
/bin/bash: steps/compute_cmvn_stats.sh: No such file or directory
/bin/bash: utils/fix_data_dir.sh: No such file or directory
/bin/bash: steps/make_mfcc.sh: No such file or directory
/bin/bash: steps/compute_cmvn_stats.sh: No such file or directory
/bin/bash: utils/fix_data_dir.sh: No such file or directory


In [30]:
!ln -s /kaggle/input/kaldi-librispeech-model/exp/nnet3_cleaned/ exp/nnet3_cleaned
!ln -s /kaggle/input/kaldi-librispeech-model/exp/chain_cleaned/ exp/chain_cleaned

In [31]:
%env nspk=$(wc -l <data/test-clean_hires/spk2utt)
!steps/online/nnet2/extract_ivectors_online.sh --cmd "$train_cmd" --nj "${nspk}" data/test-clean_hires exp/nnet3_cleaned/extractor exp/nnet3_cleaned_out/ivectors_test-clean_hires
%env nspk=$(wc -l <data/test-other_hires/spk2utt)
!steps/online/nnet2/extract_ivectors_online.sh --cmd "$train_cmd" --nj "${nspk}" data/test-other_hires exp/nnet3_cleaned/extractor exp/nnet3_cleaned_out/ivectors_test-other_hires

env: nspk=$(wc -l <data/test-clean_hires/spk2utt)
/bin/bash: steps/online/nnet2/extract_ivectors_online.sh: No such file or directory
env: nspk=$(wc -l <data/test-other_hires/spk2utt)
/bin/bash: steps/online/nnet2/extract_ivectors_online.sh: No such file or directory


In [32]:
!cat exp/nnet3_cleaned_out/ivectors_test-clean_hires/log/extract_ivectors.*.log

cat: 'exp/nnet3_cleaned_out/ivectors_test-clean_hires/log/extract_ivectors.*.log': No such file or directory


In [33]:
!ls -lR /kaggle/working/data

/kaggle/working/data:
total 0


In [34]:
!ls utils

utils
